### Summary
- Cropsy is analysing recent scan data from a specific block within a vineyard. This block has
been scanned twice
- We have the block and row geometries as well as the processed vine stats and locations for the first scan.
- The second scan was done on a different day and has
similar vine information as the first scan.
- The goal is to create a pipeline to clean the data from the second scan, match it with the first, and create a summary analysis for internal use and customer delivery.

### Task
- A dataset with block and row geometries are provided. As well as vine information for
the two scans.
- Tidy the second scan and assign missing vines in the gaps.
- Match the second scan data with the previous scan by location.
- For Cropsy internal metrics, report statistics on scan quality and reproducibility.
- For customer delivery, report on vine attribute statistics and overall data quality.

### Data
- **block.geojson:** A single Polygon defining the boundaries of the block within a
vineyard. The block also contains important row spacing and vine spacing
information.
- **row.geojson:** A collection of LineString and MultiLineStrings with row ids and
numbers.
- **scan1_processed_insights.csv:** A .csv containing data of vines and posts
detected by a Cropsy tractor. As well as corresponding attributes including x, y
coordinates, time in epoch, and data. This data has been tidied, each vine is a
vine spacing away from each other, and posts are in line.
- **scan2_raw_insights.csv:** A .csv containing similar data to scan1. Except it
has not been tidied, that is, the geolocations need to be tidied and missing
vines need to be added.
- The attributes important for statistics are Cane Count, Bud Count, and
whether the vines are missing or not.

### My Code
- *zhaohan.xiong.ipynb*: This notebook to do all the analysis
- *requirements.txt*: Dependencies for this project
- *pyvenv.cfg*: Python version
```
pip install -r requirements.txt
```

In [9]:
# Python 3.10.3
# dependencies
import json
import numpy as np
import pandas as pd
import geopandas as gpd

# Shapely

In [13]:
rows = json.loads("data/rows.geojson")
block = json.loads("data/block.geojson")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [10]:
scan1 = pd.read_csv("data/scan1_processed_insights.csv")
scan2 = pd.read_csv("data/scan2_raw_insights.csv")

In [12]:
scan1.head()

,UniqueId,EpochTime,RowId,BlockId,x,y,Type,Missing,Cane Count,Bud Count,VineQuality,VelocityKph
0,214200785-0-1691687031,1.691687e+09,66055,102,0.000000e+00,49.890566,POST,0,NaN,NaN,100.0,4.173664
1,214200785-7-1691687031,1.691687e+09,66055,102,0.000000e+00,57.110247,POST,0,NaN,NaN,100.0,4.352175
2,214200785-12-1691687031,1.691687e+09,66055,102,4.656613e-10,64.252164,POST,0,NaN,NaN,100.0,4.322781
3,214200785-15-1691687031,1.691687e+09,66055,102,4.656613e-10,71.510959,POST,0,NaN,NaN,100.0,4.329529
4,214200785-20-1691687031,1.691687e+09,66055,102,0.000000e+00,78.690838,POST,0,NaN,NaN,100.0,4.488187
